In [ ]:
import requests, base64
import spacy

invoke_url = "https://ai.api.nvidia.com/v1/genai/nvidia/consistory"
headers = {
    "Authorization": "Bearer YOUR-KEY",
    "Accept": "application/json",
}


nlp = spacy.load("en_core_web_sm")

def subject(sentence):
    doc = nlp(sentence)

    subject = None

    for token in doc:
        if 'subj' in token.dep_:
            subject = token.text
            break

    if not subject:
        for token in doc:
            if token.pos_ in ['NOUN', 'PRON']:
                subject = token.text
                break

    return subject if subject else "sepia"


def tokenizer(input):
    words = input.split()
    middle_index = len(words) // 2

    token1 = " ".join(words[:middle_index])
    token2 = " ".join(words[middle_index:])

    return token1, token2

def main():
    while True:
        user_prompt = input("Enter your prompt (type 'exit' to quit): ")

        if user_prompt.lower() == 'exit':
            break


        prompt1, prompt2 = tokenizer(user_prompt)
        token1 = subject(prompt1)
        token2 = subject(prompt2)
        print(prompt1 + ' ' + token1)
        print(prompt2 + ' ' + token2)

        payload = {
            "mode": 'init',
            "subject_prompt": user_prompt,
            "subject_tokens": [token1, token2],
            "subject_seed": 44,
            "style_prompt": "A photo of",
            "scene_prompt1": prompt1,
            "scene_prompt2": prompt2,
            "negative_prompt": "",
            "cfg_scale": 5,
            "same_initial_noise": False
        }

        response = requests.post(invoke_url, headers=headers, json=payload)
        response.raise_for_status()

        data = response.json()

        for idx, img_data in enumerate(data['artifacts']):
          img_base64 = img_data["base64"]
          img_bytes = base64.b64decode(img_base64)
          with open(f'{idx}.jpg', "wb") as f:
              f.write(img_bytes)

if __name__ == "__main__":
    main()